In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import time
plt.style.use('fivethirtyeight')
#color_pad = plt.rcParams['axes.prod_cycle'].by_key()['color']
from fredapi import Fred

In [2]:
#set Fred api key
fred_key = 'fred_key'
fred = Fred(fred_key)

In [3]:
sp_search = fred.search('S&P', order_by='realtime_end')

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:
sp500 = fred.get_series('SP500')

In [ ]:
sp500.plot(figsize=(10,5), title='S&P 500')

In [ ]:
unemp_rates = fred.search('unemployment')

In [ ]:
unrate = fred.get_series('UNRATE')

In [ ]:
unrate.plot()

In [ ]:
unemp_df=fred.search('unemployment rate state', filter=('frequency', 'Monthly'))

In [ ]:
fig, ax = plt.subplots()

In [ ]:
unemp_df = unemp_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
unemp_df = unemp_df.loc[unemp_df['title'].str.contains('Unemployment Rate')]
unemp_df['title'] = unemp_df['title'].str.replace('Unemployment Rate in ', '').to_dict()
unemp_df

In [ ]:
all_results = []

for myid in unemp_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
unemp_results = pd.concat(all_results, axis=1)

In [ ]:
unemp_results = unemp_results.drop('UNRATE', axis=1)
unemp_results = unemp_results.drop('M0892AUSM156SNBR', axis=1)
unemp_results = unemp_results.drop('M0892BUSM156SNBR', axis=1)
unemp_results = unemp_results.drop('LRHUTTTTUSM156S', axis=1)
unemp_results = unemp_results.drop('U2RATE', axis=1)
unemp_results = unemp_results[unemp_results.columns.drop(list(unemp_results.filter(regex='LASMT')))]
unemp_results = unemp_results[unemp_results.columns.drop(list(unemp_results.filter(regex='LRUN')))]
unemp_states = unemp_results[unemp_results.columns.drop(list(unemp_results.filter(regex='LNS')))]
unemp_states

In [ ]:
id_to_state = unemp_df['title']
unemp_states.columns = [id_to_state[c] for c in unemp_states.columns]

In [ ]:
unemp_states = unemp_states.dropna()

In [ ]:
px.line(unemp_states)

In [ ]:
#locate April 1st row and transform the data set
ax = unemp_states.loc[unemp_states.index == '2020-05-01'].T \
    .sort_values('2020-05-01') \
    .plot(kind='barh', figsize=(8, 12), width=0.7, edgecolor='black',
          title='Unemployment Rate by State, May 2020')
ax.legend().remove()
ax.set_xlabel('% Unemployed')
plt.show()

# Participation Rate

In [ ]:
df = fred.search('participation rate state', filter=('frequency','Monthly'))
df = df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')

part_id_to_state = df['title'].str.replace('Labor Force Participation Rate for ','').to_dict()
all_results = []
for myid in df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
    time.sleep(0.1) # Don't request to fast and get blocked
part_states = pd.concat(all_results, axis=1)
part_states.columns = [part_id_to_state[c] for c in part_states.columns]

In [ ]:
#Rename DC
unemp_states = unemp_states.rename(columns={'the District of Columbia':'District Of Columbia'})

In [ ]:
fig, ax = plt.subplots()
ax2 = ax.twinx()
unemp_states.query('index>=1975')['North Dakota'] \
    .plot(ax=ax2, label='Unemployment', color='blue')
part_states.query('index>=1975')['North Dakota'] \
    .plot(ax=ax2, label='Participation', color='red')
ax2.grid(False)
ax2.set_title('North Dakota')
ax2.set_ylabel('unemployment rate')

# Participation Rate vs Unemployment Rate

In [ ]:
fig, axs = plt.subplots(10, 5, figsize=(30, 30), sharex=True)
axs = axs.flatten()

i = 0
for state in unemp_states.columns:
    if state in ["District Of Columbia","Puerto Rico"]:
        continue
    ax2 = axs[i].twinx()
    unemp_states.query('index >= 2020 and index < 2022')[state] \
        .plot(ax=axs[i], label='Unemployment')
    part_states.query('index >= 2020 and index < 2022')[state] \
        .plot(ax=ax2, label='Participation')
    ax2.grid(False)
    axs[i].set_title(state)
    i += 1
plt.tight_layout()
plt.show()
